<a href="https://colab.research.google.com/github/chocozzang/My_Python_Practice/blob/main/%EC%9E%91%EC%97%85_2%EC%9C%A0%ED%98%95_%EB%A1%9C%EC%A7%80%EC%8A%A4%ED%8B%B1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
data = pd.read_csv('/content/sample_data/breast-cancer-wisconsin.csv', encoding="utf-8")

data = data.loc[data['bare_nucleoli'].str.isnumeric()] # bare_nucleoli 만 object이므로 숫자형인 튜플만 받아옴 isnumeric 사용

x = data[data.columns[1:10]] # 특성치
y = data[["class"]]          # 레이블 데이터 셋

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, stratify = y, random_state = 42)
# train set 과 test set 을 7:3(기본 비율)로 설정
# y 범주의 비율에 따라 분리되도록 stratify = y로 설정

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses
133,3,1,1,1,2,1,2,2,1
6,1,1,1,1,2,10,3,1,1
0,5,1,1,1,2,1,3,1,1
278,1,1,1,1,2,1,3,1,1
58,5,2,3,1,6,10,5,1,1
...,...,...,...,...,...,...,...,...,...
530,7,4,4,3,4,10,6,9,1
222,4,1,1,3,1,5,2,1,1
326,10,1,1,1,2,10,5,4,1
11,2,1,1,1,2,1,2,1,1


clump_thickness       int64
size_uniformity       int64
shape_uniformity      int64
marginal_adhesion     int64
epithelial_size       int64
bare_nucleoli        object
bland_chromatin       int64
normal_nucleoli       int64
mitoses               int64
dtype: object


In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(x_train)
x_scaled_train = scaler.transform(x_train)
x_scaled_test = scaler.transform(x_test)

# MinMaxScaler : 최소 0 / 최대 1로 정규화
# 이때, x_train의 최대, 최소를 기준으로 fit하여 x_train과 x_test셋을 변환(transform)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_scaled_train, y_train) # 로지스틱 회귀 모델에 fitting
pred_train = model.predict(x_scaled_train) # 예측치를 pred_train에 저장
model.score(x_scaled_train, y_train) # 훈련데이터가 예측치와 얼마나 부합하는지를 score로 확인함

0.97265625

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_train = confusion_matrix(y_train, pred_train) # 오차 행렬을 찾음
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[328   5]
 [  9 170]]


훈련데이터 오차행렬 : 
 [[328   5]  # 정상(0) 중 5명이 오차
 [  9 170]]  # 환자(1) 중 9명이 오차

In [ ]:
from sklearn.metrics import classification_report

cfreport_train = classification_report(y_train, pred_train) # 여러 평가지표를 레포트로 나타냄
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           2       0.97      0.98      0.98       333
           4       0.97      0.95      0.96       179

    accuracy                           0.97       512
   macro avg       0.97      0.97      0.97       512
weighted avg       0.97      0.97      0.97       512



In [ ]:
pred_test = model.predict(x_scaled_test)
model.score(x_scaled_test, y_test) # 테스트데이터가 예측치와 얼마나 부합하는지 확인

0.9590643274853801

In [ ]:
from sklearn.metrics import confusion_matrix

confusion2_train = confusion_matrix(y_test, pred_test) # 오차행렬 y와 예측치를 비교
print(confusion2_train)

[[106   5]
 [  2  58]]


In [ ]:
from sklearn.metrics import classification_report

cfreport2 = classification_report(y_test, pred_test) # 분류예측 레포트는 y와 예측치를 비교
print("분류예측 레포트 : \n", cfreport2)

테스트데이터 오차행렬 : 
               precision    recall  f1-score   support

           2       0.98      0.95      0.97       111
           4       0.92      0.97      0.94        60

    accuracy                           0.96       171
   macro avg       0.95      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



------------------------GRID SEARCH--------------------------

In [ ]:
param_grid = {'C' : [0.001, 0.01, 0.1, 1, 10, 100]}

from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv = 5)
grid_search.fit(x_scaled_train, y_train) # 훈련 데이터로 fitting -> 점수를 테스트 데이터로

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]})

In [ ]:
print("Best Parameter : {}".format(grid_search.best_params_))  # 그리드 서치에서의 최적 파라미터
print("Best Score : {}".format(grid_search.best_score_))       # 그리드 서치에서의 최적 점수
print("TestSet    Score :     {:.4f}".format(grid_search.score(x_scaled_test, y_test))) # 그리드 서치의 정확도

Best Parameter : {'C': 10}
Best Score : 0.972606129830573
TestSet    Score :     0.9591


-----------------------RANDOM SEARCH-----------------------------

In [ ]:
from scipy.stats import randint

param_distributs = {'C' : randint(low=1, high=100)}

from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(LogisticRegression(), param_distributions=param_distributs, n_iter = 100, cv = 5)
random_search.fit(x_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=LogisticRegression(), n_iter=100,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7f1bf4786650>})

In [ ]:
print("Best Parameter : {}".format(random_search.best_params_))  # 그리드 서치에서의 최적 파라미터
print("Best Score : {}".format(random_search.best_score_))       # 그리드 서치에서의 최적 점수
print("TestSet    Score :     {:.4f}".format(random_search.score(x_scaled_test, y_test))) # 그리드 서치의 정확도

Best Parameter : {'C': 12}
Best Score : 0.9745478774033887
TestSet    Score :     0.9591
